In [1]:
import time
import random

import argparse

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


########################################################################


In [2]:
#############################################################################
def read_data(data_path, show=False):
    """Read the specified data (KQBF problem)"""
    # data_path = "./instances/kqbf/kqbf020"

    with open(data_path, "r") as file:
        lines = file.readlines()

    N = int(lines[0].strip())
    W = int(lines[1].strip())
    w = list(map(int, lines[2].strip().split()))
    A = []

    for i in range(N):
        row = list(map(int, lines[i + 3].strip().split()))
        A.append(row)

    max_length = max(len(sublist) for sublist in A)
    for sublist in A:
        sublist.extend([0] * (max_length - len(sublist)))

    matrix = np.zeros((max_length, max_length))
    for i in range(max_length):
        for j in range(i + 1):
            matrix[i][j] = A[i][j]
            matrix[j][i] = A[i][j]

    if show:
        print("N:", N)
        print("W:", W)
        print("w:", w)
        print("A:")
        for row in A:
            print(row)

    return N, W, np.array(w), np.array(matrix)

#############################################################################
# Definindo a função objetivo QBF
def objective_function(x, A):
    return np.dot(x, np.dot(A, x.T))

# Definindo a restrição do problema
def constraint(x, w, W, show=False):
    result = np.dot(x, w)
    if show:
        print(f"<x|w> <= W?")
        print(f"<{x}|{w}> = {result}  <= {W}??? {result <= W}")
    return result <= W

def all_population(n):
    """Initialize the population with all possible combinations"""
    if n == 0:
        return [[]]
    else:
        smaller = all_population(n - 1)
        return [x + [0] for x in smaller] + [x + [1] for x in smaller]
    
def initialize_population(all_pop):

    return np.array(all_pop)

In [19]:
problem_instance = "kqbf020"
N, W, w, A = read_data(data_path=f"./instances/kqbf/{problem_instance}", show=False)
w = np.array(w)

In [20]:
# import numpy as np

# def greedy_knapsack(weights, A, capacity):
#     # calculate objective function for each item
#     objectives = [objective_function(np.eye(1, len(weights), i)[0], A) for i in range(len(weights))]
    
#     # sort items by decreasing order of objective function
#     items = sorted(list(range(len(weights))), key=lambda i: -objectives[i])
    
#     solution = [0]*len(weights)
#     total_weight = 0
    
#     # while the smallest item can still fit in
#     for i in items:
#         if total_weight + weights[i] <= capacity:
#             # add item with largest objective function value
#             solution[i] = 1
#             # update remaining weight
#             total_weight += weights[i]
    
#     # check the constraint
#     contraint_checker = constraint(solution, weights, capacity)
#     if not contraint_checker:
#         print("The solution does not satisfy the constraint.")
#         return None
    
#     # return solution
#     return solution


# # Call the function and print the solution
# solution = greedy_knapsack(w, A, W)
# print(f"The solution is {solution}")


In [21]:
# def greedy_knapsack(weights, A, capacity, beta, alpha, gamma):
#     # calculate objective function for each item
#     objectives = [objective_function(np.eye(1, len(weights), i)[0], A) for i in range(len(weights))]
    
#     # sort items by decreasing order of objective function
#     items = sorted(list(range(len(weights))), key=lambda i: -objectives[i])
    
#     # create a Restricted Candidate List (RCL) with the top beta items
#     rcl = items[:beta]
    
#     # filter the RCL to only include items that are not more than alpha% away from the best element
#     rcl = [i for i in rcl if objectives[i] >= (1 - alpha) * objectives[rcl[0]]]
#     # filter the RCL to only include items above an absolute threshold of gamma
#     rcl = [i for i in rcl if objectives[i] >= gamma]
    
#     solution = [0]*len(weights)
#     total_weight = 0
    
#     # while the smallest item can still fit in
#     for i in rcl:
#         if total_weight + weights[i] <= capacity:
#             # add item with largest objective function value
#             solution[i] = 1
#             # update remaining weight
#             total_weight += weights[i]
    
#     # check the constraint
#     if not constraint(solution, weights, capacity):
#         print("The solution does not satisfy the constraint.")
#         return None, None
    
#     # calculate the value of the objective function for the current solution
#     objective_value = objective_function(np.array(solution), A)
    
#     # return solution
#     return solution, objective_value


# # Call the function and print the solution
# beta = 2
# alpha = 0.1
# gamma = 0.5

# solution, objective_value = greedy_knapsack(w, A, W, beta, alpha, gamma)
# print(f"The solution is {solution}\n...objective value is {objective_value}")

In [35]:
def select_element(candidate_list):
    # Select an element from the candidate list with equal probability
    return random.choice(candidate_list)

def make_candidate_list(weights, A, solution, beta, alpha, gamma):
    # Calculate objective function for each item
    objectives = [objective_function(np.eye(1, len(weights), i)[0], A) for i in range(len(weights))]

    # Sort items by decreasing order of objective function
    items = sorted(list(range(len(weights))), key=lambda i: -objectives[i])

    # Create a Restricted Candidate List (RCL) with the top beta items
    rcl = items[:beta]

    # Filter the RCL to only include items that are not more than alpha% away from the best element
    rcl = [i for i in rcl if objectives[i] >= (1 - alpha) * objectives[rcl[0]]]

    # Filter the RCL to only include items above an absolute threshold of gamma
    rcl = [i for i in rcl if objectives[i] >= gamma]

    # Filter the RCL to only include items that are not already in the solution
    rcl = [i for i in rcl if solution[i] == 0]

    return rcl

def construct_solution(weights, A, capacity, beta, alpha, gamma):
    # Initialize an empty solution
    solution = [0]*len(weights)
    total_weight = 0  # Keep track of the total weight of the solution

    # While solution construction is not finished
    while sum(solution) != len(weights):
        # Make a candidate list
        candidate_list = make_candidate_list(weights, A, solution, beta, alpha, gamma)

        # If the candidate list is empty, break the loop
        if not candidate_list:
            break

        # Select an element from the candidate list
        s = select_element(candidate_list)

        # Check if adding the selected item would exceed the capacity
        if total_weight + weights[s] > capacity:
            continue

        # Add the selected element to the solution
        solution[s] = 1
        total_weight += weights[s]  # Update the total weight of the solution

        # Change the greedy function (This is not specified in the pseudocode, so I'll leave it as a comment)
        # change_greedy_function()

    # Return the solution
    return solution

def local_search_best_improving(solution, weights, A, capacity):
    """Best-improving local search strategy"""
    improved = True
    while improved:
        improved = False
        best_new_solution = solution.copy()
        best_new_cost = objective_function(solution, A)
        for i in range(len(solution)):
            new_solution = solution.copy()
            new_solution[i] = (new_solution[i] + 1) % 2
            # Check if the new solution is feasible
            if constraint(new_solution, weights, capacity):
                new_cost = objective_function(new_solution, A)
                if new_cost > best_new_cost:
                    best_new_solution = new_solution
                    best_new_cost = new_cost
                    improved = True
        solution = best_new_solution
    return solution, best_new_cost

def grasp(weights, A, capacity, beta, alpha, gamma, max_iterations):
    # Initialize the best solution to an empty solution with an objective value of 0
    current_best = ([0]*len(weights), float("-inf"))
    cost_history = []

    # Repeat until a certain condition is met (e.g., a maximum number of iterations)
    for _ in range(max_iterations):
        # Construct a solution
        solution = construct_solution(weights, A, capacity, beta, alpha, gamma)
        solution_array = np.array(solution)  # Convert the solution to a numpy array
        objective_value = objective_function(solution_array, A)

        # Apply local search to the solution
        new_solution, new_objective_value = local_search_best_improving(solution_array, weights, A, capacity)

        # If the new solution is better than the current best, update the current best
        if new_objective_value > current_best[1]:
            current_best = (new_solution.tolist(), new_objective_value)

            print(new_objective_value)

        cost_history.append(new_objective_value)

            

    # Return the best solution found
    return current_best

# def local_search(solution, weights, A, capacity):
#     # This is a placeholder for the local search function.
#     # You should replace this with your actual local search implementation.
#     return solution, objective_function(solution, A)

max_iterations = 100
beta = 5
alpha = 0.5
gamma = 0.2

solution, objective_value = grasp(w, A, W, beta, alpha, gamma, max_iterations)
print(f"The solution is {solution}\n...objective value is {objective_value}")


160.0
The solution is [1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0]
...objective value is 160.0


In [23]:
constraint(solution, w, W)

True

In [24]:



def local_search_best_improving(solution, A):
    """Best-improving local search strategy"""
    improved = True
    while improved:
        improved = False
        best_new_solution = solution.copy()
        best_new_cost = objective_function(solution, A)
        for i in range(len(solution)):
            for j in range(len(solution)):
                if i != j:
                    new_solution = solution.copy()
                    new_solution[i] = (new_solution[i] + 1) % 2
                    new_solution[j] = (new_solution[j] + 1) % 2
                    new_cost = objective_function(new_solution, A)
                    if new_cost > best_new_cost:
                        best_new_solution = new_solution
                        best_new_cost = new_cost
                        improved = True
        solution = best_new_solution
    return solution, best_new_cost

def local_search_first_improving(solution, A):
    """First-improving local search strategy"""
    improved = True
    while improved:
        improved = False
        for i in range(len(solution)):
            for j in range(len(solution)):
                if i != j:
                    new_solution = solution.copy()
                    new_solution[i] = (new_solution[i] + 1) % 2
                    new_solution[j] = (new_solution[j] + 1) % 2
                    new_cost = objective_function(new_solution, A)
                    if new_cost > objective_function(solution, A):
                        solution = new_solution
                        improved = True
                        break  # Stop at the first improving move
            if improved:
                break
    return solution

# Local search with pertubation costs
def perturb_cost(A, percent_change=0.1):
    """
    Perturba os custos da matriz de custos.
    
    Parameters:
        A (list of lists): A matriz de custos original.
        percent_change (float): Percentual máximo de alteração permitido nos custos. O valor padrão é 0.1, o que corresponde a 10% de mudança.

    Returns:
        perturbed_A (list of lists): A matriz de custos perturbada.
    """
    num_nodes = len(A)
    perturbed_A = [[0] * num_nodes for _ in range(num_nodes)]

    for i in range(num_nodes):
        for j in range(num_nodes):
            change = A[i][j] * percent_change
            perturbed_A[i][j] = A[i][j] + random.uniform(-change, change)

    return np.array(perturbed_A)

def local_search(solution, A, improvement_strategy="best-improving", perturb=False, percent_change=0.1):
    
    # Aplicar perturbação de custo, se necessário
    if perturb:
        A = perturb_cost(A, percent_change)

    if improvement_strategy == "best-improving":
        solution = local_search_best_improving(solution, A)
    elif improvement_strategy == "first-improving":
        solution = local_search_best_improving(solution, A)
    return solution


def grasp_qbf(N, A, w, W, 
              alpha=0.2, 
              iterations=100, 
              seed=42, 
              improvement_strategy="best-improving", 
              perturb=False, 
              percent_change=0.1, 
              minutes=None):
    
    best_solution = None
    best_cost = float('-inf')

    counter = 0
    # logs_df = pd.DataFrame()
    # iterations_lst, objective_functions = [], []

    if minutes is None:

        for i in range(iterations):
            solution_indices = greedy_randomized_construction_with_constraint(alpha, A, w, W, seed=seed+i)
            if not solution_indices:
                continue  # No feasible solution
            solution_indices = local_search(solution_indices, A, improvement_strategy=improvement_strategy, perturb=perturb, percent_change=percent_change)
            current_cost = objective_function(solution_indices, A)
            if current_cost > best_cost:
                best_solution = solution_indices
                best_cost = current_cost
            
            # iterations_lst.append(i)
            # objective_functions.append(current_cost)
                    
    else:

        secs = minutes * 60
        start_time = time.time()
        while (time.time() - start_time) < secs:

            solution_indices = greedy_randomized_construction_with_constraint(alpha, A, w, W, seed=seed+counter)
            if not solution_indices:
                continue  # No feasible solution
            solution_indices = local_search(solution_indices, 
                                            A, 
                                            improvement_strategy=improvement_strategy, 
                                            perturb=perturb, percent_change=percent_change)
            current_cost = objective_function(solution_indices, A)
            if current_cost > best_cost:
                best_solution = solution_indices
                best_cost = current_cost
                
                # print(f"Iter {i}...Solution inside GRASP: {solution_indices}")
                # print(f"......binarized: {get_binary_solution(solution_indices, N)}\n")

            # iterations_lst.append(counter)
            # objective_functions.append(current_cost)

            counter += 1
    
    # logs_df["iteration"] = iterations_lst
    # logs_df["objective_function"] = objective_functions
            
    best_solution_binarized = get_binary_solution(best_solution, N)

    return best_solution_binarized, best_cost #, logs_df

# #############################################################################
# if __name__ == "__main__":

#     # HOW TO RUN? Call inside a terminal: python3 GRASP_KQBF.py kqbf020 --minutes 30 (example)

#     # Using minutes as counter
#     print()
#     print("Reading the data...")

#     parser = argparse.ArgumentParser(description="Process the problem instance.")
#     parser.add_argument("problem_instance", type=str, help="Problem instance string")
#     parser.add_argument("--minutes", type=float, default=1, help="Duration in minutes (default: 1)")
#     args = parser.parse_args()

#     problem_instance = args.problem_instance
#     N, W, w, A = read_data(data_path=f"./instances/kqbf/{problem_instance}", show=False)
#     minutes = args.minutes

#     print()
#     print("Starting the search...")
#     print()
    
#     config_dict = {
#         "GRASP_alpha1_first": grasp_qbf(N, A, w, W, 
#                                         alpha=0.3, 
#                                         iterations=10, 
#                                         improvement_strategy="first-improving",
#                                         perturb=False, 
#                                         percent_change=0.1, 
#                                         minutes=minutes), 
#         "GRASP_alpha2": grasp_qbf(N, A, w, W, 
#                                     alpha=0.5, 
#                                     iterations=10, 
#                                     improvement_strategy="first-improving",
#                                     perturb=False, 
#                                     percent_change=0.1, 
#                                     minutes=minutes),

#         "GRASP_best": grasp_qbf(N, A, w, W, 
#                                     alpha=0.0, 
#                                     iterations=10, 
#                                     improvement_strategy="best-improving",
#                                     perturb=False, 
#                                     percent_change=0.1, 
#                                     minutes=minutes),

#         "GRASP_cost_pertubation": grasp_qbf(N, A, w, W, 
#                                     alpha=0.0, 
#                                     iterations=10, 
#                                     improvement_strategy="best-improving",
#                                     perturb=True, 
#                                     percent_change=0.33, 
#                                     minutes=minutes),

#     }

#     results_df = pd.DataFrame()
#     config_lst, solutions_lst, costs_lst, bag_values_lst = [], [], [], []

    
#     for config, grasp_call in config_dict.items():
#         print(f"...Starting process for config {config}")

#         best_solution, best_cost = grasp_call

#         config_lst.append(config)
#         costs_lst.append(best_cost)
#         solutions_lst.append(best_solution)
#         bag_values_lst.append(constraint(best_solution, w))

#     print()
#     print("Finished the process...")

#     results_df["configuration"] = config_lst
#     results_df["costs"] = costs_lst
#     results_df["constraint_value"] = bag_values_lst
#     results_df["best_solutions"] = solutions_lst

#     results_df.to_csv(f"./outputs/{problem_instance}_result.csv", 
#                       index=False, 
#                       sep="|")
    


